# Construcción del JSON de referencia J1939-71 para RAG

Este cuaderno carga el PDF 'J1939-71.pdf', realiza una extracción básica de texto (para validar acceso al documento) y construye programáticamente el archivo `j1939_71_reference.json` con la estructura consensuada para el RAG y el enlace a DBC.

In [1]:
# Rutas de entrada/salida
from pathlib import Path
PDF_PATH = Path(r'c:\Users\henry\Documents\GitHub\Proyecto_Integrador_Grupo7_IBM\Entrega4_Precision_Semantica_y_Contextualizacion_Eventos_Vehiculares\J1939-71.pdf')
OUTPUT_JSON = Path(r'c:\Users\henry\Documents\GitHub\Proyecto_Integrador_Grupo7_IBM\Entrega4_Precision_Semantica_y_Contextualizacion_Eventos_Vehiculares\j1939_71_reference.json')

print('PDF existe:', PDF_PATH.exists(), str(PDF_PATH))
print('Salida JSON en:', str(OUTPUT_JSON))

PDF existe: True c:\Users\henry\Documents\GitHub\Proyecto_Integrador_Grupo7_IBM\Entrega4_Precision_Semantica_y_Contextualizacion_Eventos_Vehiculares\J1939-71.pdf
Salida JSON en: c:\Users\henry\Documents\GitHub\Proyecto_Integrador_Grupo7_IBM\Entrega4_Precision_Semantica_y_Contextualizacion_Eventos_Vehiculares\j1939_71_reference.json


In [2]:
# Instalar dependencia mínima si hace falta: pdfplumber
import importlib, sys, subprocess
def ensure_package(pkg):
    try:
        return importlib.import_module(pkg)
    except ImportError:
        print(f'Instalando {pkg}...')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])
        return importlib.import_module(pkg)

pdfplumber = ensure_package('pdfplumber')
print('pdfplumber versión:', getattr(pdfplumber, '__version__', 'desconocida'))

pdfplumber versión: 0.11.7


In [3]:
# Carga básica del PDF y extracción limitada (validación)
extracted_preview = ''
if PDF_PATH.exists():
    with pdfplumber.open(PDF_PATH) as pdf:
        pages_to_read = min(3, len(pdf.pages))
        texts = []
        for i in range(pages_to_read):
            txt = pdf.pages[i].extract_text() or ''
            texts.append(txt)
        extracted_preview = '\n---\n'.join(texts)
        print(f'Páginas leídas: {pages_to_read} / {len(pdf.pages)}')
        print('Previsualización (primeros 500 caracteres):')
        print(extracted_preview[:500])
else:
    print('Advertencia: PDF no encontrado en la ruta indicada.')

Páginas leídas: 3 / 379
Previsualización (primeros 500 caracteres):
REV.
SURFACE J 1939-71 DEC2003
®
VEHICLE
Issued 1994-08
RECOMMENDED
Revised 2003-12
PRACTICE
Su perseding J1939-71 AUG2002
Vehicle Application Layer — J1939-71 (through December 2001)
Foreword
This document has also changed to comply with the SAE Technical Standards Board format. Definitions
have changed to Section 3 and Abbreviations to Section 4. All other section numbers have changed
accordingly. This series of SAE Recommended Practices has been developed by the SAE Truck and
Bus Control and 


In [3]:
# Parsear el PDF completo y construir el JSON desde definiciones SPN
import re, json, datetime

# Utilidades de parsing

def _num(s: str):
    s2 = s.strip().replace(",", "")
    try:
        return float(s2)
    except Exception:
        return None


def _clean_text(s: str) -> str:
    import re as _re
    return _re.sub(r"\s+", " ", s or "").strip()


def read_pdf_all_text(path: Path) -> str:
    try:
        import pdfplumber  # type: ignore
    except Exception:
        return ""
    if not path.exists():
        return ""
    parts = []
    try:
        with pdfplumber.open(path) as pdf:
            for pg in pdf.pages:
                parts.append(pg.extract_text() or "")
        return "\n\n".join(parts)
    except Exception:
        return ""


def parse_spn_blocks(full_text: str):
    if not full_text:
        return []
    block_re = re.compile(
        r"(?is)\bspn\s*([0-9]{3,6})\s*-\s*(.+?)\s*-\s*(.+?)\s*"
        r"Data\s+Length:\s*([^\n]+)\n"
        r"Resolution:\s*([^\n]+)\n"
        r"Data\s+Range:\s*([^\n]+)\n"
        r"Type:\s*([^\n]+)\n"
        r"Suspect\s+Parameter\s+Number:\s*([0-9]{3,6})\n"
        r"Parameter\s+Group\s+Number:\s*([^\n]+?)\s*(?=\n\s*spn\s*[0-9]{3,6}\s*-|\Z)"
    )
    items = []
    for m in block_re.finditer(full_text):
        spn_id = int(m.group(1))
        name = _clean_text(m.group(2))
        desc = _clean_text(m.group(3))
        data_len_line = m.group(4).strip()
        resolution_line = m.group(5).strip()
        range_line = m.group(6).strip()
        type_line = m.group(7).strip()
        pgn_line = m.group(9).strip()

        bits_len = None
        mlen = re.search(r"([0-9.,]+)\s*(bytes?|bits?)", data_len_line, re.I)
        if mlen:
            n = _num(mlen.group(1)) or 0
            unit = (mlen.group(2) or "").lower()
            bits_len = int(round(n * 8)) if "byte" in unit else int(round(n))

        scale = None
        units = None
        offset = None
        mres = re.search(r"([\-0-9.,]+)\s*([^\s,]+)?\s*/\s*bit.*?([\-0-9.,]+)\s*offset", resolution_line, re.I)
        if mres:
            scale = _num(mres.group(1))
            units = (mres.group(2) or "").strip() or None
            offset = _num(mres.group(3))

        rmin = None
        rmax = None
        units_range = None
        mrange = re.search(r"([\-0-9.,]+)\s*to\s*([\-0-9.,]+)\s*([^\s%°A-Za-z]*)?\s*([%°A-Za-z]+)?", range_line, re.I)
        if mrange:
            rmin = _num(mrange.group(1))
            rmax = _num(mrange.group(2))
            units_range = (mrange.group(4) or mrange.group(3) or "").strip() or None
        if not units and units_range:
            units = units_range

        if re.search(r"measured", type_line, re.I):
            spn_type = "Medido"
        elif re.search(r"status|state", type_line, re.I):
            spn_type = "Estado"
        else:
            spn_type = _clean_text(type_line)

        pgns = [int(x) for x in re.findall(r"\[\s*([0-9]{5})\s*\]", pgn_line)]

        items.append({
            "spn": spn_id,
            "name": name,
            "description": desc,
            "units": units,
            "data_length_bits": bits_len,
            "resolution": scale,
            "offset": offset,
            "range": {"min": rmin, "max": rmax} if (rmin is not None or rmax is not None) else None,
            "type": spn_type,
            "pgns": pgns,
            "states": None,
            "notes": None,
        })
    return items


now_iso = datetime.datetime.utcnow().replace(microsecond=0).isoformat() + 'Z'
full_text = read_pdf_all_text(PDF_PATH)
spn_items = parse_spn_blocks(full_text)
print('SPNs extraídos:', len(spn_items))

# Construir PGNs mínimos a partir de asociaciones
pgn_map = {}
for spn in spn_items:
    for pgn in spn.get('pgns', []) or []:
        if pgn not in pgn_map:
            pgn_map[pgn] = {
                'pgn': pgn,
                'name': None,
                'data_length_bytes': None,
                'data_page': None,
                'pdu_format': None,
                'pdu_specific': None,
                'default_priority': None,
                'transmission_rate': None,
                'signals': [],
                'notes': 'Estructura mínima inferida; completar con DBC.'
            }
        pgn_map[pgn]['signals'].append({
            'spn': spn.get('spn'),
            'name': spn.get('name'),
            'start_bit': None,
            'length': spn.get('data_length_bits'),
            'endianness': 'Intel',
            'units': spn.get('units'),
            'scale': spn.get('resolution'),
            'offset': spn.get('offset'),
            'min': (spn.get('range') or {}).get('min'),
            'max': (spn.get('range') or {}).get('max'),
            'dbc_link': {'message': None, 'signal': None}
        })

pgn_list = list(pgn_map.values())

# Ensamblar el JSON
data = {
  'metadata': {
    'standard': 'SAE J1939-71',
    'description': 'Capa de Aplicación J1939: directrices, definiciones de SPN/PGN y campos para enlazar con DBC. Archivo de referencia para modelos RAG.',
    'source_document': 'Entrega4_Precision_Semantica_y_Contextualizacion_Eventos_Vehiculares/J1939-71.pdf',
    'created_at': now_iso,
    'language': 'es',
    'schema_version': '1.0.0'
  },
  'guidelines': {
    'signal_characterization': {
      'latency_recommendations': 'Minimizar la latencia entre adquisición y transmisión. Documentar latencia y jitter esperados por parámetro.',
      'time_base': 'Algunos PGN dependen del ángulo del cigüeñal/velocidad de motor (sincronizados a fase).'
    },
    'message_format': {
      'pgn': 'El Número de Grupo de Parámetros (PGN) identifica el mensaje y su contenido lógico.',
      'data_types': [
        'ASCII (ISO/IEC 8859-1 / Latin-1)',
        'Datos escalados (factor + offset)',
        'Estados de función (bitfields/enumeraciones)'
      ],
      'byte_order': 'Little-endian (LSB primero) salvo indicación contraria en el SPN/PGN.',
      'measured_vs_state': "Cada SPN se clasifica como 'Medido' o 'Estado'."
    },
    'charset': {
      'name': 'ISO/IEC 8859-1 (Latin-1)',
      'notes': 'Para campos de texto/ASCII se recomienda Latin-1. Limitar a ASCII imprimible cuando aplique.'
    },
    'parameter_ranges': {
      'general': "Definir rango válido, valores para 'no disponible' y 'error/indeterminado' según el tipo de dato.",
      'common_conventions': [
        "Para 1 byte: 0..254 suelen mapear a valores/estados válidos, y 255 (0xFF) suele indicar 'no disponible'.",
        "Para múltiples bytes escalados: usar todo el rango útil, reservando el máximo para 'no disponible' cuando aplique.",
        "Los valores exactos de 'error'/'no disponible' pueden ser específicos del SPN; ver la definición del SPN."
      ]
    },
    'slot_allocation': {
      'scaling': 'Definir resolución (factor), offset y límites. Preferir escalas que maximicen el uso del rango sin saturación.',
      'limits_and_offsets': 'Explicitar min/max físicos y el offset aplicado.'
    },
    'adding_parameters_to_groups': {
      'grouping': 'Agregar nuevos parámetros a PGNs existentes cuando sea lógico; mantener posiciones y alineación de bytes/bits estables.',
      'compatibility': 'Evitar romper compatibilidad hacia atrás; documentar cambios en posiciones/tamaños.'
    },
    'transmission_rates': {
      'nominal_rates': ['10 ms', '20 ms', '100 ms', '1 s', 'a demanda'],
      'notes': [
        'Elegir tasas periódicas vs. por evento según la dinámica del parámetro.',
        'Mensajes dependientes del cigüeñal se envían conforme a la velocidad del motor.'
      ]
    },
    'naming_conventions': {
      'multi_component': "Para parámetros con múltiples instancias, añadir índice/ubicación (ej.: 'temperatura_escape_1', 'temperatura_escape_2').",
      'clarity': 'Mantener nombres consistentes con J1939 para facilitar el cruce con DBC/SPN.'
    },
    'multi_source_notes': {
      'source_priority': 'Si un parámetro tiene múltiples fuentes (ECU diferentes), definir reglas de prioridad y reconciliación.',
      'provenance': 'Registrar la ECU origen (Source Address) y la estrategia de selección.'
    }
  },
  'dbc_linkage': {
    'can_id_composition': 'ID 29 bits = Prioridad(3) | Reservado(1) | Data Page(1) | PDU Format(8) | PDU Specific(8) | Source Address(8)',
    'fields_mapping': {
      'dbc_message': {
        'name': 'Nombre del mensaje en DBC (habitualmente el nombre del PGN)',
        'can_id': 'Identificador CAN extendido de 29 bits',
        'pgn': 'Número PGN',
        'source_address': '0-255',
        'pdu_format': '0-255',
        'pdu_specific': '0-255',
        'priority': '0-7'
      },
      'dbc_signal': {
        'name': 'Nombre de la señal en DBC (alineado con nombre SPN)',
        'spn': 'Número SPN',
        'start_bit': 'Bit de inicio (convención Intel/little-endian)',
        'length': 'Longitud en bits',
        'endianness': 'Intel (little-endian) salvo indicación contraria',
        'scale': 'Factor (resolución)',
        'offset': 'Offset',
        'min': 'Mínimo físico',
        'max': 'Máximo físico',
        'units': 'Unidades'
      }
    },
    'matching_strategy': [
      'Cruzar por SPN (clave primaria semántica).',
      'Verificar unidades, resolución y offset.',
      'Usar nombre del PGN y señal como respaldo, cuidando sinónimos.',
      'Confirmar posiciones (start_bit/length) desde el DBC, no desde este archivo.'
    ]
  },
  'spns': spn_items,
  'pgns': pgn_list,
  'traceability': {
    'sections': {
      '5.1': 'Directrices generales (latencia, formato, charset, rangos, slots, agrupación, tasas, nomenclatura, multi-fuente).',
      '5.2': 'Definiciones de parámetros (SPN): nombre, descripción, longitud, escala, rango, tipo, SPN, PGN.',
      '5.3': 'Definiciones de grupos (PGN): nombre, tasa, longitud, DP/PDU, prioridad, lista de SPNs con posiciones.'
    }
  }
}

C:\Users\henry\AppData\Local\Temp\ipykernel_41312\1902645085.py:112: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now_iso = datetime.datetime.utcnow().replace(microsecond=0).isoformat() + 'Z'


KeyboardInterrupt: 

In [ ]:
# Parser streaming rápido con PyMuPDF (fitz) independiente (no depende de 'data' previa)
# - Procesa el PDF por chunks y muestra progreso
# - Limita páginas con MAX_PAGES para pruebas rápidas (None para procesar todo)

import importlib, sys, subprocess, re, datetime
from pathlib import Path as _PathAlias  # para evitar conflictos de nombres

# Parámetros de rendimiento
MAX_PAGES = 50  # ponlo en None para procesar todo el PDF
CHUNK_PAGES = 10
ENGINE = 'fitz'  # 'fitz' (PyMuPDF) o 'pdfplumber'

# Asegurar PyMuPDF si se va a usar
if ENGINE == 'fitz':
    try:
        import fitz  # PyMuPDF
    except Exception:
        print('Instalando PyMuPDF…')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pymupdf'])
        import fitz

# Fallback a pdfplumber para lectura por página si fitz no está disponible
try:
    import pdfplumber as _pp
except Exception:
    print('Instalando pdfplumber…')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pdfplumber'])
    import pdfplumber as _pp


def iter_pages_text_nb(pdf_path: _PathAlias, engine: str = 'fitz'):
    if engine == 'fitz':
        try:
            with fitz.open(pdf_path) as doc:
                for page in doc:
                    yield page.get_text('text') or ''
            return
        except Exception as e:
            print('Aviso: PyMuPDF falló; usando pdfplumber…', e)
    with _pp.open(pdf_path) as pdf:
        for p in pdf.pages:
            yield p.extract_text() or ''


pattern = re.compile(
    r"(?is)\bspn\s*([0-9]{3,6})\s*-\s*(.+?)\s*-\s*(.+?)\s*"
    r"Data\s+Length:\s*([^\n]+)\n"
    r"Resolution:\s*([^\n]+)\n"
    r"Data\s+Range:\s*([^\n]+)\n"
    r"Type:\s*([^\n]+)\n"
    r"Suspect\s+Parameter\s+Number:\s*([0-9]{3,6})\n"
    r"Parameter\s+Group\s+Number:\s*([^\n]+?)\s*(?=\n\s*spn\s*[0-9]{3,6}\s*-|\Z)"
)


def _num(s: str):
    try:
        return float(s.strip().replace(',', ''))
    except Exception:
        return None


# Streaming parse con progreso y límite de páginas
spn_items_stream = []
buf = []
lead = ''
page_count = 0
for text in iter_pages_text_nb(PDF_PATH, engine=ENGINE):
    page_count += 1
    buf.append(text)
    if len(buf) >= CHUNK_PAGES:
        chunk = lead + "\n\n".join(buf)
        lead = chunk[-2000:]
        for m in pattern.finditer(chunk):
            spn_id = int(m.group(1))
            name = _clean_text(m.group(2))
            desc = _clean_text(m.group(3))
            data_len_line = m.group(4).strip()
            resolution_line = m.group(5).strip()
            range_line = m.group(6).strip()
            type_line = m.group(7).strip()
            pgn_line = m.group(9).strip()
            # Data Length
            bits_len = None
            mlen = re.search(r"([0-9.,]+)\s*(bytes?|bits?)", data_len_line, re.I)
            if mlen:
                n = _num(mlen.group(1)) or 0
                unit = (mlen.group(2) or '').lower()
                bits_len = int(round(n*8)) if 'byte' in unit else int(round(n))
            # Resolution
            scale = None; units = None; offset = None
            mres = re.search(r"([\-0-9.,]+)\s*([^\s,]+)?\s*/\s*bit.*?([\-0-9.,]+)\s*offset", resolution_line, re.I)
            if mres:
                scale = _num(mres.group(1))
                units = (mres.group(2) or '').strip() or None
                offset = _num(mres.group(3))
            # Range
            rmin = None; rmax = None; units_range = None
            mrange = re.search(r"([\-0-9.,]+)\s*to\s*([\-0-9.,]+)\s*([^\s%°A-Za-z]*)?\s*([%°A-Za-z]+)?", range_line, re.I)
            if mrange:
                rmin = _num(mrange.group(1)); rmax = _num(mrange.group(2))
                units_range = (mrange.group(4) or mrange.group(3) or '').strip() or None
            if not units and units_range:
                units = units_range
            # Type
            if re.search(r'measured', type_line, re.I):
                spn_type = 'Medido'
            elif re.search(r'status|state', type_line, re.I):
                spn_type = 'Estado'
            else:
                spn_type = _clean_text(type_line)
            # PGNs
            pgns = [int(x) for x in re.findall(r"\[\s*([0-9]{5})\s*\]", pgn_line)]
            spn_items_stream.append({
                'spn': spn_id,
                'name': name,
                'description': desc,
                'units': units,
                'data_length_bits': bits_len,
                'resolution': scale,
                'offset': offset,
                'range': {'min': rmin, 'max': rmax} if (rmin is not None or rmax is not None) else None,
                'type': spn_type,
                'pgns': pgns,
                'states': None,
                'notes': None,
            })
        print(f"Procesadas {page_count} páginas; SPNs: {len(spn_items_stream)}")
        buf = []
    if MAX_PAGES is not None and page_count >= MAX_PAGES:
        print(f"Límite MAX_PAGES alcanzado: {MAX_PAGES}")
        break

# Procesar resto del buffer
if buf:
    chunk = lead + "\n\n".join(buf)
    for m in pattern.finditer(chunk):
        spn_id = int(m.group(1))
        name = _clean_text(m.group(2))
        desc = _clean_text(m.group(3))
        data_len_line = m.group(4).strip()
        resolution_line = m.group(5).strip()
        range_line = m.group(6).strip()
        type_line = m.group(7).strip()
        pgn_line = m.group(9).strip()
        bits_len = None
        mlen = re.search(r"([0-9.,]+)\s*(bytes?|bits?)", data_len_line, re.I)
        if mlen:
            n = _num(mlen.group(1)) or 0
            unit = (mlen.group(2) or '').lower()
            bits_len = int(round(n*8)) if 'byte' in unit else int(round(n))
        scale = None; units = None; offset = None
        mres = re.search(r"([\-0-9.,]+)\s*([^\s,]+)?\s*/\s*bit.*?([\-0-9.,]+)\s*offset", resolution_line, re.I)
        if mres:
            scale = _num(mres.group(1)); units = (mres.group(2) or '').strip() or None; offset = _num(mres.group(3))
        rmin = None; rmax = None; units_range = None
        mrange = re.search(r"([\-0-9.,]+)\s*to\s*([\-0-9.,]+)\s*([^\s%°A-Za-z]*)?\s*([%°A-Za-z]+)?", range_line, re.I)
        if mrange:
            rmin = _num(mrange.group(1)); rmax = _num(mrange.group(2)); units_range = (mrange.group(4) or mrange.group(3) or '').strip() or None
        if not units and units_range:
            units = units_range
        if re.search(r'measured', type_line, re.I):
            spn_type = 'Medido'
        elif re.search(r'status|state', type_line, re.I):
            spn_type = 'Estado'
        else:
            spn_type = _clean_text(type_line)
        pgns = [int(x) for x in re.findall(r"\[\s*([0-9]{5})\s*\]", pgn_line)]
        spn_items_stream.append({
            'spn': spn_id,
            'name': name,
            'description': desc,
            'units': units,
            'data_length_bits': bits_len,
            'resolution': scale,
            'offset': offset,
            'range': {'min': rmin, 'max': rmax} if (rmin is not None or rmax is not None) else None,
            'type': spn_type,
            'pgns': pgns,
            'states': None,
            'notes': None,
        })

print(f"Total SPNs extraídos (stream): {len(spn_items_stream)}")

# Reconstruir pgn_list a partir de asociaciones SPN->PGN (sin start_bit; se completa con DBC)
pgn_map = {}
for spn in spn_items_stream:
    for pgn in spn.get('pgns', []) or []:
        if pgn not in pgn_map:
            pgn_map[pgn] = {
                'pgn': pgn,
                'name': None,
                'data_length_bytes': None,
                'data_page': None,
                'pdu_format': None,
                'pdu_specific': None,
                'default_priority': None,
                'transmission_rate': None,
                'signals': [],
                'notes': 'Estructura mínima inferida; completar con DBC.'
            }
        pgn_map[pgn]['signals'].append({
            'spn': spn.get('spn'),
            'name': spn.get('name'),
            'start_bit': None,
            'length': spn.get('data_length_bits'),
            'endianness': 'Intel',
            'units': spn.get('units'),
            'scale': spn.get('resolution'),
            'offset': spn.get('offset'),
            'min': (spn.get('range') or {}).get('min'),
            'max': (spn.get('range') or {}).get('max'),
            'dbc_link': {'message': None, 'signal': None}
        })

pgn_list = list(pgn_map.values())

# Construir data completo e independiente
now_iso = datetime.datetime.utcnow().replace(microsecond=0).isoformat() + 'Z'
metadata_block = {
  'standard': 'SAE J1939-71',
  'description': 'Capa de Aplicación J1939: directrices, definiciones de SPN/PGN y campos para enlazar con DBC. Archivo de referencia para modelos RAG.',
  'source_document': 'Entrega4_Precision_Semantica_y_Contextualizacion_Eventos_Vehiculares/J1939-71.pdf',
  'created_at': now_iso,
  'language': 'es',
  'schema_version': '1.0.0'
}

guidelines_block = {
  'signal_characterization': {
    'latency_recommendations': 'Minimizar la latencia entre adquisición y transmisión. Documentar latencia y jitter esperados por parámetro.',
    'time_base': 'Algunos PGN dependen del ángulo del cigüeñal/velocidad de motor (sincronizados a fase).'
  },
  'message_format': {
    'pgn': 'El Número de Grupo de Parámetros (PGN) identifica el mensaje y su contenido lógico.',
    'data_types': [
      'ASCII (ISO/IEC 8859-1 / Latin-1)',
      'Datos escalados (factor + offset)',
      'Estados de función (bitfields/enumeraciones)'
    ],
    'byte_order': 'Little-endian (LSB primero) salvo indicación contraria en el SPN/PGN.',
    'measured_vs_state': "Cada SPN se clasifica como 'Medido' o 'Estado'."
  },
  'charset': {
    'name': 'ISO/IEC 8859-1 (Latin-1)',
    'notes': 'Para campos de texto/ASCII se recomienda Latin-1. Limitar a ASCII imprimible cuando aplique.'
  },
  'parameter_ranges': {
    'general': "Definir rango válido, valores para 'no disponible' y 'error/indeterminado' según el tipo de dato.",
    'common_conventions': [
      "Para 1 byte: 0..254 suelen mapear a valores/estados válidos, y 255 (0xFF) suele indicar 'no disponible'.",
      "Para múltiples bytes escalados: usar todo el rango útil, reservando el máximo para 'no disponible' cuando aplique.",
      "Los valores exactos de 'error'/'no disponible' pueden ser específicos del SPN; ver la definición del SPN."
    ]
  },
  'slot_allocation': {
    'scaling': 'Definir resolución (factor), offset y límites. Preferir escalas que maximicen el uso del rango sin saturación.',
    'limits_and_offsets': 'Explicitar min/max físicos y el offset aplicado.'
  },
  'adding_parameters_to_groups': {
    'grouping': 'Agregar nuevos parámetros a PGNs existentes cuando sea lógico; mantener posiciones y alineación de bytes/bits estables.',
    'compatibility': 'Evitar romper compatibilidad hacia atrás; documentar cambios en posiciones/tamaños.'
  },
  'transmission_rates': {
    'nominal_rates': ['10 ms', '20 ms', '100 ms', '1 s', 'a demanda'],
    'notes': [
      'Elegir tasas periódicas vs. por evento según la dinámica del parámetro.',
      'Mensajes dependientes del cigüeñal se envían conforme a la velocidad del motor.'
    ]
  },
  'naming_conventions': {
    'multi_component': "Para parámetros con múltiples instancias, añadir índice/ubicación (ej.: 'temperatura_escape_1', 'temperatura_escape_2').",
    'clarity': 'Mantener nombres consistentes con J1939 para facilitar el cruce con DBC/SPN.'
  },
  'multi_source_notes': {
    'source_priority': 'Si un parámetro tiene múltiples fuentes (ECU diferentes), definir reglas de prioridad y reconciliación.',
    'provenance': 'Registrar la ECU origen (Source Address) y la estrategia de selección.'
  }
}


dbc_linkage_block = {
  'can_id_composition': 'ID 29 bits = Prioridad(3) | Reservado(1) | Data Page(1) | PDU Format(8) | PDU Specific(8) | Source Address(8)',
  'fields_mapping': {
    'dbc_message': {
      'name': 'Nombre del mensaje en DBC (habitualmente el nombre del PGN)',
      'can_id': 'Identificador CAN extendido de 29 bits',
      'pgn': 'Número PGN',
      'source_address': '0-255',
      'pdu_format': '0-255',
      'pdu_specific': '0-255',
      'priority': '0-7'
    },
    'dbc_signal': {
      'name': 'Nombre de la señal en DBC (alineado con nombre SPN)',
      'spn': 'Número SPN',
      'start_bit': 'Bit de inicio (convención Intel/little-endian)',
      'length': 'Longitud en bits',
      'endianness': 'Intel (little-endian) salvo indicación contraria',
      'scale': 'Factor (resolución)',
      'offset': 'Offset',
      'min': 'Mínimo físico',
      'max': 'Máximo físico',
      'units': 'Unidades'
    }
  },
  'matching_strategy': [
    'Cruzar por SPN (clave primaria semántica).',
    'Verificar unidades, resolución y offset.',
    'Usar nombre del PGN y señal como respaldo, cuidando sinónimos.',
    'Confirmar posiciones (start_bit/length) desde el DBC, no desde este archivo.'
  ]
}

traceability_block = {
  'sections': {
    '5.1': 'Directrices generales (latencia, formato, charset, rangos, slots, agrupación, tasas, nomenclatura, multi-fuente).',
    '5.2': 'Definiciones de parámetros (SPN): nombre, descripción, longitud, escala, rango, tipo, SPN, PGN.',
    '5.3': 'Definiciones de grupos (PGN): nombre, tasa, longitud, DP/PDU, prioridad, lista de SPNs con posiciones.'
  }
}

# Ensamblar data final
data = {
  'metadata': metadata_block,
  'guidelines': guidelines_block,
  'dbc_linkage': dbc_linkage_block,
  'spns': spn_items_stream,
  'pgns': pgn_list,
  'traceability': traceability_block
}

print('data listo con:', len(data.get('spns', [])), 'SPNs y', len(data.get('pgns', [])), 'PGNs inferidos')

Procesadas 10 páginas; SPNs: 0
Procesadas 20 páginas; SPNs: 0
Procesadas 30 páginas; SPNs: 0
Procesadas 40 páginas; SPNs: 13
Procesadas 50 páginas; SPNs: 60
Procesadas 60 páginas; SPNs: 84
Procesadas 70 páginas; SPNs: 98
Procesadas 80 páginas; SPNs: 112
Procesadas 20 páginas; SPNs: 0
Procesadas 30 páginas; SPNs: 0
Procesadas 40 páginas; SPNs: 13
Procesadas 50 páginas; SPNs: 60
Procesadas 60 páginas; SPNs: 84
Procesadas 70 páginas; SPNs: 98
Procesadas 80 páginas; SPNs: 112
Procesadas 90 páginas; SPNs: 116
Procesadas 100 páginas; SPNs: 120
Procesadas 110 páginas; SPNs: 136
Procesadas 120 páginas; SPNs: 161
Procesadas 130 páginas; SPNs: 201
Procesadas 140 páginas; SPNs: 237
Procesadas 150 páginas; SPNs: 274
Procesadas 160 páginas; SPNs: 306
Procesadas 170 páginas; SPNs: 351
Procesadas 180 páginas; SPNs: 375
Procesadas 190 páginas; SPNs: 384
Procesadas 200 páginas; SPNs: 402
Procesadas 90 páginas; SPNs: 116
Procesadas 100 páginas; SPNs: 120
Procesadas 110 páginas; SPNs: 136
Procesadas 120 

NameError: name 'data' is not defined

In [ ]:
# Guardar el JSON en disco
OUTPUT_JSON.parent.mkdir(parents=True, exist_ok=True)
with open(OUTPUT_JSON, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)
print('JSON escrito en:', OUTPUT_JSON)

In [ ]:
# Carga de verificación
with open(OUTPUT_JSON, 'r', encoding='utf-8') as f:
    loaded = json.load(f)
print('Claves de nivel superior:', list(loaded.keys()))
print('SPNs de ejemplo:', [s['spn'] for s in loaded.get('spns', [])])
print('PGNs de ejemplo:', [p['pgn'] for p in loaded.get('pgns', [])])